<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

# 2.3 Data Access and Basic Processing

<img src="https://openeo.org/images/openeo_logo.png"
     alt="openEO logo"
     style="float: center; margin-right: 10px; max-height: 100px;"/>

## Filter Operators with openEO

When interacting with large data collections, it is necessary to keep in mind that it's not possible to load everything!

Therefore, we always have to define our requirements in advance and apply them to the data using filter operators.

Let's start again with the same sample data from the Sentinel-2 STAC Collection with an additional filter.

### Properties Filter

When working with optical data like Sentinel-2, most of the times we would like to discard cloudy acquisitions as soon as possible.

We can do it using a property filter: in this case we want to keep only the acquisitions with less than 50% cloud coverage.

In [1]:
properties = {"eo:cloud_cover": dict(lt=50)}

In [2]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent = {"west": 11.1, "east": 11.5, "south": 46.1, "north": 46.5}
temporal_extent = ["2023-04-01T00:00:00Z","2023-11-30T00:00:00Z"]

datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    temporal_extent=temporal_extent,
                    properties=properties)
datacube.execute()

<xarray.DataArray 'stackstac-1124b32c76286561d7a7e9aed71246e9' (time: 41,
                                                                band: 32,
                                                                y: 4534, x: 3209)> Size: 153GB
dask.array<getitem, shape=(41, 32, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 328B 2023-...
    id                                       (time) <U24 4kB 'S2A_32TPS_20230...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 26kB 6.611e+05 ... 6...
  * y                                        (y) float64 36kB 5.153e+06 ... 5...
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

### Temporal Filter

To filter along time the data collection with openEO, we can use the `filter_temporal` process.

In [3]:
temporal_extent = ["2023-05-10T00:00:00Z","2023-06-30T00:00:00Z"]
temporal_slice = datacube.filter_temporal(temporal_extent)
temporal_slice.execute()

<xarray.DataArray 'stackstac-1124b32c76286561d7a7e9aed71246e9' (time: 7,
                                                                band: 32,
                                                                y: 4534, x: 3209)> Size: 26GB
dask.array<getitem, shape=(7, 32, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 56B 2023-0...
    id                                       (time) <U24 672B 'S2A_32TPS_2023...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 26kB 6.611e+05 ... 6...
  * y                                        (y) float64 36kB 5.153e+06 ... 5...
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

After running the previous cell, it is visible that the result has less elements (or labels) in the temporal dimension `time`.

Additionally, the size of the selected data reduced a lot.

**Quiz hint: look carefully at the dimensions of the resulting datacube!**

### Spatial Filter

To slice along the spatial dimensions the data collection with openEO, we can use `filter_bbox` or `filter_spatial` processes.

The `filter_bbox` process is used with a set of coordinates:

In [4]:
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}
spatial_slice = datacube.filter_bbox(spatial_extent)
spatial_slice.execute()

<xarray.DataArray 'stackstac-1124b32c76286561d7a7e9aed71246e9' (time: 41,
                                                                band: 32,
                                                                y: 489, x: 1145)> Size: 6GB
dask.array<getitem, shape=(41, 32, 489, 1145), dtype=float64, chunksize=(1, 1, 382, 860), chunktype=numpy.ndarray>
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 328B 2023-...
    id                                       (time) <U24 4kB 'S2A_32TPS_20230...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 9kB 6.733e+05 ... 6....
  * y                                        (y) float64 4kB 5.153e+06 ... 5....
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

**Quiz hint: look carefully at the dimensions of the loaded datacube!**

### Bands Filter

To slice along the bands dimension, keeping only the necessary bands, we can use the `filter_bands` process.

In [5]:
bands = ["red","green","blue"]
bands_slice = datacube.filter_bands(bands)
bands_slice.execute()

<xarray.DataArray 'stackstac-1124b32c76286561d7a7e9aed71246e9' (time: 41,
                                                                band: 3,
                                                                y: 4534, x: 3209)> Size: 14GB
dask.array<getitem, shape=(41, 3, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 328B 2023-...
    id                                       (time) <U24 4kB 'S2A_32TPS_20230...
  * band                                     (band) <U12 144B 'red' ... 'blue'
  * x                                        (x) float64 26kB 6.611e+05 ... 6...
  * y                                        (y) float64 36kB 5.153e+06 ... 5...
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    ...                                       ...
    raster:bands                             (band) object 24B [{'nodata': 0,...
    gsd                                      (band) object 24B 10 10 10
    common_name                              (band) object 24B 'red' ... 'blue'
    center_wavelength                        (band) object 24B 0.665 0.56 0.49
    full_width_half_max                      (band) object 24B 0.038 0.045 0.098
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0